**Bayesian Optimization for Evolutionary Scenarios**

Bayesian Optimization for all 9 selection scenarios

**Number of Trials: 50**

**Number of Simulations Used: 40,000**

**GPU Used: A100**

In [ ]:

!git clone https://github.com/Djinho/EvoNet-CNN-Insight.git

# Navigate to the cloned repository
%cd EvoNet-CNN-Insight/Optimisation_notebooks

Cloning into 'EvoNet-CNN-Insight'...
remote: Enumerating objects: 6054, done.
remote: Counting objects: 100% (1675/1675), done.
remote: Compressing objects: 100% (1039/1039), done.
remote: Total 6054 (delta 722), reused 1525 (delta 618), pack-reused 4379 (from 1)
Receiving objects: 100% (6054/6054), 8.08 GiB | 46.71 MiB/s, done.
Resolving deltas: 100% (2313/2313), done.
Updating files: 100% (216/216), done.
/content/EvoNet-CNN-Insight/Optimisation_notebooks


In [2]:

!pip install optuna
!pip install optuna-integration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 7.4 MB/s eta 0:00:00


In [3]:
import optuna
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, callbacks
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import logging
import os
import time
import gzip
import skimage.transform
from optuna.integration import TFKerasPruningCallback

In [4]:

%run -i ../ImaGene.py


In [ ]:
# Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='Ancient_Scenarios_AM.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/Ancient_moderate/AM'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///am_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=50)  # Set to 50 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function


Starting optimization for /content/EvoNet-CNN-Insight/model_training/Ancient_moderate/AM


[I 2024-08-11 21:04:32,025] A new study created in RDB with name: no-name-c492fb7f-dc7c-4412-9dbd-8d7338bfea6c


Trial 0 started with hyperparameters: num_layers=6, filters=[55, 60, 87, 50, 64, 33], kernel_size=2, l1=0.00035664681070305104, l2=0.0005729901632281326, dense_units=193, dropout_rate=0.3107556174793292, learning_rate=1.690623670619887e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 13s 79ms/step - accuracy: 0.4912 - loss: 2.2807 - val_accuracy: 0.5175 - val_loss: 2.2617
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4865 - loss: 2.2570 - val_accuracy: 0.5200 - val_loss: 2.2383


/usr/local/lib/python3.10/dist-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5119 - loss: 2.2325 - val_accuracy: 0.4725 - val_loss: 2.2162
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5042 - loss: 2.2096 - val_accuracy: 0.5150 - val_loss: 2.1921
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5149 - loss: 2.1866 - val_accuracy: 0.5225 - val_loss: 2.1693
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5344 - loss: 2.1640 - val_accuracy: 0.5275 - val_loss: 2.1474
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5222 - loss: 2.1422 - val_accuracy: 0.5962 - val_loss: 2.1251
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5661 - loss: 2.1198 - val_accuracy: 0.5987 - val_loss: 2.1034
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5483 - loss: 2.0986 - val_accuracy: 0.6137 - val_loss: 2.0821
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6110 - loss: 2.0821


...

Trial 48 started with hyperparameters: num_layers=3, filters=[40, 124, 103], kernel_size=3, l1=2.195655020140089e-06, l2=0.0006963405631647386, dense_units=239, dropout_rate=0.13236113984711123, learning_rate=1.4527206048438652e-05
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5064 - loss: 0.8293

[I 2024-08-12 02:30:23,438] Trial 48 pruned. Trial was pruned at epoch 0.


Trial 49 started with hyperparameters: num_layers=4, filters=[97, 66, 43, 56], kernel_size=2, l1=1.045876041342991e-05, l2=0.002171021539639442, dense_units=210, dropout_rate=0.4140624319100813, learning_rate=3.339705968715223e-05
48/50 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5085 - loss: 1.1111

[I 2024-08-12 02:31:55,453] Trial 49 pruned. Trial was pruned at epoch 0.


Best trial:
Value: 0.7225000262260437
Params: 
    num_layers: 5
    filters_0: 65
    filters_1: 128
    filters_2: 77
    filters_3: 48
    filters_4: 127
    kernel_size: 2
    l1: 5.39171365329603e-06
    l2: 0.000508098182990793
    dense_units: 122
    dropout_rate: 0.44775481260866545
    learning_rate: 0.00023021217142648836
Elapsed time: 19644.33 seconds


Ancient and strong selection

In [ ]:

# Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='Ancient_Scenarios_AS.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/Ancient_strong/AS'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///as_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=50)  # Set to 50 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function


Starting optimization for /content/EvoNet-CNN-Insight/model_training/Ancient_strong/AS


[I 2024-08-13 11:10:17,548] A new study created in RDB with name: no-name-da99b1f4-10c0-4dcf-80b8-0fe09ecf6472


Trial 0 started with hyperparameters: num_layers=5, filters=[118, 124, 38, 78, 78], kernel_size=3, l1=1.549449854388779e-05, l2=0.0001894902426840597, dense_units=121, dropout_rate=0.11205384804668804, learning_rate=1.251314385156787e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 30s 262ms/step - accuracy: 0.5012 - loss: 0.8696 - val_accuracy: 0.4875 - val_loss: 0.8680
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 129ms/step - accuracy: 0.5223 - loss: 0.8673 - val_accuracy: 0.4913 - val_loss: 0.8658


/usr/local/lib/python3.10/dist-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 130ms/step - accuracy: 0.5214 - loss: 0.8652 - val_accuracy: 0.5487 - val_loss: 0.8626
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.5261 - loss: 0.8624 - val_accuracy: 0.5100 - val_loss: 0.8605
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 130ms/step - accuracy: 0.5391 - loss: 0.8604 - val_accuracy: 0.5100 - val_loss: 0.8586
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.5493 - loss: 0.8578 - val_accuracy: 0.4950 - val_loss: 0.8568
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.5269 - loss: 0.8556 - val_accuracy: 0.5312 - val_loss: 0.8512
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.5156 - loss: 0.8538 - val_accuracy: 0.4925 - val_loss: 0.8547
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.5262 - loss: 0.8517 - val_accuracy: 0.6150 - val_loss: 0.8471
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.6127 - loss: 0.8466


...

Trial 48 started with hyperparameters: num_layers=3, filters=[104, 57, 91], kernel_size=3, l1=0.0036212070150548168, l2=0.000795735286931679, dense_units=207, dropout_rate=0.2741834321935408, learning_rate=4.224083472922052e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 203ms/step - accuracy: 0.5081 - loss: 12.6624 - val_accuracy: 0.4825 - val_loss: 12.0933
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.4893 - loss: 11.9087 - val_accuracy: 0.4800 - val_loss: 11.3578
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.5079 - loss: 11.1772 - val_accuracy: 0.5550 - val_loss: 10.6452
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.5155 - loss: 10.4737 - val_accuracy: 0.5825 - val_loss: 9.9605
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.5359 - loss: 9.7950 - val_accuracy: 0.5088 - val_loss: 9.3027
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.5180 - loss: 9.1434 - val_accuracy: 0.4950 - val_loss: 8.6705
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 105ms/step - accuracy: 0.5198 - 

[I 2024-08-13 16:59:35,818] Trial 48 finished with value: 0.5805000066757202 and parameters: {'num_layers': 3, 'filters_0': 104, 'filters_1': 57, 'filters_2': 91, 'kernel_size': 3, 'l1': 0.0036212070150548168, 'l2': 0.000795735286931679, 'dense_units': 207, 'dropout_rate': 0.2741834321935408, 'learning_rate': 4.224083472922052e-05}. Best is trial 4 with value: 0.6287500262260437.


Trial 49 started with hyperparameters: num_layers=4, filters=[88, 85, 122, 40], kernel_size=3, l1=8.139097571408362e-06, l2=7.442998453007265e-05, dense_units=250, dropout_rate=0.20477975640930332, learning_rate=6.074789931733123e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.4854 - loss: 0.7657

[I 2024-08-13 17:01:21,934] Trial 49 pruned. Trial was pruned at epoch 0.


Best trial:
Value: 0.6287500262260437
Params: 
    num_layers: 4
    filters_0: 44
    filters_1: 83
    filters_2: 72
    filters_3: 64
    kernel_size: 3
    l1: 0.00013537781122606584
    l2: 0.0037394012860948295
    dense_units: 167
    dropout_rate: 0.18690845934507774
    learning_rate: 1.0525370626878665e-05
Elapsed time: 21065.31 seconds


ancient and weak selection

In [ ]:
#Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='Ancient_Scenarios_AW.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/Ancient_weak/AW'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///aw_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=50)  # Set to 50 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function


Starting optimization for /content/EvoNet-CNN-Insight/model_training/Ancient_weak/AW


[I 2024-08-14 08:55:52,119] A new study created in RDB with name: no-name-224fb6af-34df-4042-ad31-4c6510227cdb


Trial 0 started with hyperparameters: num_layers=3, filters=[124, 62, 50], kernel_size=2, l1=0.005142999155128057, l2=0.0003479960392214099, dense_units=138, dropout_rate=0.3836690939826398, learning_rate=2.266424435926498e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 174ms/step - accuracy: 0.5249 - loss: 11.6111 - val_accuracy: 0.4837 - val_loss: 11.4201
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.5515 - loss: 11.3526 - val_accuracy: 0.6675 - val_loss: 11.1614


/usr/local/lib/python3.10/dist-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 105ms/step - accuracy: 0.6028 - loss: 11.0977 - val_accuracy: 0.5163 - val_loss: 10.9154
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.5997 - loss: 10.8572 - val_accuracy: 0.6525 - val_loss: 10.6800
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.6613 - loss: 10.6126 - val_accuracy: 0.7325 - val_loss: 10.4296
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.6921 - loss: 10.3853 - val_accuracy: 0.6313 - val_loss: 10.2179
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.6333 - loss: 10.1750 - val_accuracy: 0.6438 - val_loss: 10.0483
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.6742 - loss: 9.9645 - val_accuracy: 0.6888 - val_loss: 9.8127
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.6859 - loss: 9.7593 - val_accuracy: 0.6600 - val_loss: 9.6163
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6324 - loss: 9.6236


...

Trial 48 started with hyperparameters: num_layers=3, filters=[103, 89, 67], kernel_size=2, l1=5.010392531286037e-05, l2=2.8662140441190135e-05, dense_units=174, dropout_rate=0.1521269218011923, learning_rate=0.00016437147576239271
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.5037 - loss: 0.8568

[I 2024-08-14 14:09:39,501] Trial 48 pruned. Trial was pruned at epoch 0.


Trial 49 started with hyperparameters: num_layers=4, filters=[49, 62, 82, 39], kernel_size=2, l1=0.003987074659485699, l2=4.657020954827509e-06, dense_units=189, dropout_rate=0.38592846434041694, learning_rate=3.343661598875031e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 12s 115ms/step - accuracy: 0.5189 - loss: 10.4689 - val_accuracy: 0.6425 - val_loss: 10.2440
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.5739 - loss: 10.1689 - val_accuracy: 0.6612 - val_loss: 9.9472
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.6409 - loss: 9.8741 - val_accuracy: 0.5213 - val_loss: 9.6658
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.6204 - loss: 9.5980 - val_accuracy: 0.6700 - val_loss: 9.3984
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.6850 - loss: 9.3259 - val_accuracy: 0.7425 - val_loss: 9.1200
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.7069 - loss: 9.0705 - val_accuracy: 0.6250 - val_loss: 8.8905
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.6660 - loss: 

[I 2024-08-14 14:23:53,271] Trial 49 finished with value: 0.6812499761581421 and parameters: {'num_layers': 4, 'filters_0': 49, 'filters_1': 62, 'filters_2': 82, 'filters_3': 39, 'kernel_size': 2, 'l1': 0.003987074659485699, 'l2': 4.657020954827509e-06, 'dense_units': 189, 'dropout_rate': 0.38592846434041694, 'learning_rate': 3.343661598875031e-05}. Best is trial 3 with value: 0.7682499885559082.


Best trial:
Value: 0.7682499885559082
Params: 
    num_layers: 5
    filters_0: 38
    filters_1: 104
    filters_2: 60
    filters_3: 86
    filters_4: 63
    kernel_size: 2
    l1: 5.178471083573243e-06
    l2: 0.0001446019670236893
    dense_units: 187
    dropout_rate: 0.17884975602613437
    learning_rate: 0.00015172597960847004
Elapsed time: 19682.10 seconds


Intermediate and moderate selection

In [ ]:
# Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='Intermediate_Scenarios_IM.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/intermediate_moderate/IM'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///im_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=50)  # Set to 50 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function

Starting optimization for /content/EvoNet-CNN-Insight/model_training/intermediate_moderate/IM


[I 2024-08-14 14:45:00,480] A new study created in RDB with name: no-name-e80ac72b-ff86-42f2-89a8-6571a2b7ac24


Trial 0 started with hyperparameters: num_layers=4, filters=[34, 79, 46, 128], kernel_size=2, l1=0.00028388247327834105, l2=0.000465838511968768, dense_units=84, dropout_rate=0.3457305709638745, learning_rate=0.0002843109150332335
50/50 ━━━━━━━━━━━━━━━━━━━━ 12s 104ms/step - accuracy: 0.5116 - loss: 1.4596 - val_accuracy: 0.6988 - val_loss: 1.3004
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.7010 - loss: 1.2372 - val_accuracy: 0.7775 - val_loss: 1.0551
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.7590 - loss: 1.0467 - val_accuracy: 0.7588 - val_loss: 0.9997
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.7712 - loss: 0.9679 - val_accuracy: 0.7588 - val_loss: 0.9235
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.7575 - loss: 0.9183 - val_accuracy: 0.7513 - val_loss: 0.8896
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.7655 - loss: 0.8590 - val_accuracy: 0.7663 - val_loss: 0.8361
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.7635 - loss: 0.85

[I 2024-08-14 14:59:03,911] Trial 0 finished with value: 0.7795000076293945 and parameters: {'num_layers': 4, 'filters_0': 34, 'filters_1': 79, 'filters_2': 46, 'filters_3': 128, 'kernel_size': 2, 'l1': 0.00028388247327834105, 'l2': 0.000465838511968768, 'dense_units': 84, 'dropout_rate': 0.3457305709638745, 'learning_rate': 0.0002843109150332335}. Best is trial 0 with value: 0.7795000076293945.


Trial 1 started with hyperparameters: num_layers=6, filters=[97, 75, 117, 115, 51, 100], kernel_size=2, l1=9.642593640517168e-05, l2=5.3313246214758025e-06, dense_units=135, dropout_rate=0.4852022664041491, learning_rate=0.00045541177527236374
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 190ms/step - accuracy: 0.5083 - loss: 1.3331 - val_accuracy: 0.5000 - val_loss: 1.1595
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.5067 - loss: 1.1084 - val_accuracy: 0.5200 - val_loss: 0.9711
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.5024 - loss: 0.9342 - val_accuracy: 0.5275 - val_loss: 0.8392
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.5191 - loss: 0.8177 - val_accuracy: 0.4938 - val_loss: 0.7691
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.5018 - loss: 0.7599 - val_accuracy: 0.4925 - val_loss: 0.7381
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.5074 - loss: 0.7333 - val_accuracy: 0.6225 - val_loss: 0.7216
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 

...

Trial 48 started with hyperparameters: num_layers=5, filters=[85, 53, 80, 64, 113], kernel_size=2, l1=0.0010190961383610023, l2=1.0341881678375931e-06, dense_units=155, dropout_rate=0.43080111971190316, learning_rate=9.673684306045099e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 14s 138ms/step - accuracy: 0.4974 - loss: 4.9188 - val_accuracy: 0.4825 - val_loss: 4.6340
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.5342 - loss: 4.5411 - val_accuracy: 0.7150 - val_loss: 4.2715
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.6366 - loss: 4.1853 - val_accuracy: 0.7600 - val_loss: 3.9309
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.5991 - loss: 3.8553 - val_accuracy: 0.7250 - val_loss: 3.6109
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.6903 - loss: 3.5395 - val_accuracy: 0.6575 - val_loss: 3.3486
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.7086 - loss: 3.2574 - val_accuracy: 0.6875 - val_loss: 3.1218
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.7358 - lo

[I 2024-08-14 20:58:24,457] Trial 48 finished with value: 0.7482500076293945 and parameters: {'num_layers': 5, 'filters_0': 85, 'filters_1': 53, 'filters_2': 80, 'filters_3': 64, 'filters_4': 113, 'kernel_size': 2, 'l1': 0.0010190961383610023, 'l2': 1.0341881678375931e-06, 'dense_units': 155, 'dropout_rate': 0.43080111971190316, 'learning_rate': 9.673684306045099e-05}. Best is trial 24 with value: 0.7852500081062317.


Trial 49 started with hyperparameters: num_layers=4, filters=[102, 72, 32, 91], kernel_size=2, l1=0.001802391356202278, l2=0.000520916623531664, dense_units=210, dropout_rate=0.3018479827222486, learning_rate=4.527838388547124e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 181ms/step - accuracy: 0.5019 - loss: 5.4168 - val_accuracy: 0.4825 - val_loss: 5.2659
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.5709 - loss: 5.2123 - val_accuracy: 0.7175 - val_loss: 5.0638
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.6984 - loss: 5.0180 - val_accuracy: 0.7462 - val_loss: 4.8725
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.6647 - loss: 4.8351 - val_accuracy: 0.7362 - val_loss: 4.7040
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.7269 - loss: 4.6723 - val_accuracy: 0.7600 - val_loss: 4.5524
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.7438 - loss: 4.5092 - val_accuracy: 0.6562 - val_loss: 4.4570
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.7459 - loss: 4.37

[I 2024-08-14 21:12:54,916] Trial 49 finished with value: 0.7695000171661377 and parameters: {'num_layers': 4, 'filters_0': 102, 'filters_1': 72, 'filters_2': 32, 'filters_3': 91, 'kernel_size': 2, 'l1': 0.001802391356202278, 'l2': 0.000520916623531664, 'dense_units': 210, 'dropout_rate': 0.3018479827222486, 'learning_rate': 4.527838388547124e-05}. Best is trial 24 with value: 0.7852500081062317.


Best trial:
Value: 0.7852500081062317
Params: 
    num_layers: 4
    filters_0: 101
    filters_1: 49
    filters_2: 41
    filters_3: 76
    kernel_size: 2
    l1: 0.000886865899233485
    l2: 0.00027895435203828013
    dense_units: 214
    dropout_rate: 0.2572558934185246
    learning_rate: 0.00027360324209931307
Elapsed time: 23274.69 seconds


Intermediate and strong selection

In [ ]:
# Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='intermediate_Scenarios_IS.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/intermediate_strong/IS'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///is_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=50)  # Set to 50 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function

Starting optimization for /content/EvoNet-CNN-Insight/model_training/intermediate_strong/IS


[I 2024-08-14 22:59:40,685] A new study created in RDB with name: no-name-3fad69ab-3a4c-4321-99b9-400fb09b0464


Trial 0 started with hyperparameters: num_layers=4, filters=[98, 35, 111, 123], kernel_size=3, l1=0.0002447689183653179, l2=0.0007330573344016155, dense_units=225, dropout_rate=0.1690385347736852, learning_rate=3.978281882197443e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 191ms/step - accuracy: 0.5141 - loss: 2.2066 - val_accuracy: 0.4825 - val_loss: 2.1425
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.5351 - loss: 2.1180 - val_accuracy: 0.6762 - val_loss: 2.0502
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.6584 - loss: 2.0221 - val_accuracy: 0.7025 - val_loss: 1.9153
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.7155 - loss: 1.8864 - val_accuracy: 0.7088 - val_loss: 1.8227
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.6919 - loss: 1.8216 - val_accuracy: 0.6975 - val_loss: 1.8081
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.6835 - loss: 1.7933 - val_accuracy: 0.7025 - val_loss: 1.7598
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.7379 - loss: 1.

[I 2024-08-14 23:14:26,991] Trial 0 finished with value: 0.7110000252723694 and parameters: {'num_layers': 4, 'filters_0': 98, 'filters_1': 35, 'filters_2': 111, 'filters_3': 123, 'kernel_size': 3, 'l1': 0.0002447689183653179, 'l2': 0.0007330573344016155, 'dense_units': 225, 'dropout_rate': 0.1690385347736852, 'learning_rate': 3.978281882197443e-05}. Best is trial 0 with value: 0.7110000252723694.


Trial 1 started with hyperparameters: num_layers=5, filters=[105, 107, 59, 81, 90], kernel_size=3, l1=8.066935659989512e-06, l2=0.001072238590535673, dense_units=77, dropout_rate=0.3274310397851899, learning_rate=5.846119958155314e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 216ms/step - accuracy: 0.5090 - loss: 1.1144 - val_accuracy: 0.5800 - val_loss: 1.0872
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.5243 - loss: 1.0781 - val_accuracy: 0.4800 - val_loss: 1.0542
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.6002 - loss: 1.0380 - val_accuracy: 0.6787 - val_loss: 0.9678
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.6599 - loss: 0.9540 - val_accuracy: 0.6650 - val_loss: 0.9209
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.6783 - loss: 0.9168 - val_accuracy: 0.7088 - val_loss: 0.8748
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.6939 - loss: 0.8856 - val_accuracy: 0.6475 - val_loss: 0.9217
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.7217 - 

...

Trial 48 started with hyperparameters: num_layers=3, filters=[68, 79, 92], kernel_size=3, l1=0.0001905404298438518, l2=1.889887746005714e-06, dense_units=208, dropout_rate=0.170639433275539, learning_rate=2.372536722882356e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.5186 - loss: 1.3933

[I 2024-08-15 03:47:39,286] Trial 48 pruned. Trial was pruned at epoch 0.


Trial 49 started with hyperparameters: num_layers=3, filters=[91, 108, 85], kernel_size=3, l1=7.255047932672802e-05, l2=1.7827530921362437e-05, dense_units=232, dropout_rate=0.12884885939901933, learning_rate=9.115480597644008e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.5067 - loss: 1.0772

[I 2024-08-15 03:49:22,447] Trial 49 pruned. Trial was pruned at epoch 0.


Best trial:
Value: 0.7117499709129333
Params: 
    num_layers: 6
    filters_0: 83
    filters_1: 82
    filters_2: 80
    filters_3: 118
    filters_4: 101
    filters_5: 94
    kernel_size: 3
    l1: 2.9406712943657902e-05
    l2: 2.2306887659153147e-06
    dense_units: 247
    dropout_rate: 0.3032626748136775
    learning_rate: 0.00033230173317945645
Elapsed time: 17382.02 seconds


Intermediate and weak

In [5]:

# Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='intermediate_Scenarios_IW.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/intermediate_weak/IW'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///iw_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=50)  # Set to 50 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function

Starting optimization for /content/EvoNet-CNN-Insight/model_training/intermediate_weak/IW


[I 2024-08-15 11:49:22,573] A new study created in RDB with name: no-name-7629f290-a204-468e-b142-5418b9205b6f


Trial 0 started with hyperparameters: num_layers=6, filters=[87, 45, 103, 70, 67, 48], kernel_size=2, l1=0.0019372153248872236, l2=1.6936862143720515e-06, dense_units=249, dropout_rate=0.47721701790853754, learning_rate=0.0004398097574304861
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 162ms/step - accuracy: 0.4963 - loss: 9.1235 - val_accuracy: 0.5175 - val_loss: 6.4961
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.4954 - loss: 5.7568 - val_accuracy: 0.4800 - val_loss: 3.7728


/usr/local/lib/python3.10/dist-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.5159 - loss: 3.2501 - val_accuracy: 0.4725 - val_loss: 1.9184
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.4991 - loss: 1.6133 - val_accuracy: 0.5063 - val_loss: 0.9354
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.4970 - loss: 0.8430 - val_accuracy: 0.4913 - val_loss: 0.7126
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.5096 - loss: 0.7082 - val_accuracy: 0.5050 - val_loss: 0.7036
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.5024 - loss: 0.7035 - val_accuracy: 0.4663 - val_loss: 0.7038
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.5070 - loss: 0.7034 - val_accuracy: 0.4787 - val_loss: 0.7037
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.5053 - loss: 0.7034 - val_accuracy: 0.5025 - val_loss: 0.7034
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4971 - loss: 0.7035


...

Trial 48 started with hyperparameters: num_layers=5, filters=[40, 78, 57, 80, 109], kernel_size=2, l1=0.0013176253338764328, l2=6.324867161932531e-05, dense_units=112, dropout_rate=0.18629145278274203, learning_rate=3.9039023780417255e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5206 - loss: 6.1458

[I 2024-08-15 17:01:39,723] Trial 48 pruned. Trial was pruned at epoch 0.


Trial 49 started with hyperparameters: num_layers=6, filters=[108, 86, 72, 33, 38, 40], kernel_size=3, l1=3.922379448701403e-06, l2=1.2159236265292915e-05, dense_units=168, dropout_rate=0.35155542189029126, learning_rate=5.284213613032926e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.5079 - loss: 0.7206

[I 2024-08-15 17:03:26,921] Trial 49 pruned. Trial was pruned at epoch 0.


Best trial:
Value: 0.8734999895095825
Params: 
    num_layers: 6
    filters_0: 121
    filters_1: 87
    filters_2: 54
    filters_3: 112
    filters_4: 33
    filters_5: 47
    kernel_size: 3
    l1: 5.0716695606694785e-06
    l2: 6.816909948117609e-06
    dense_units: 189
    dropout_rate: 0.4410372189591173
    learning_rate: 8.167479478069829e-05
Elapsed time: 18845.29 seconds


Recent and moderate

In [6]:

# Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='recent_Scenarios_RM.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/recent_moderate/RM'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///rm_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=50)  # Set to 50 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function

Starting optimization for /content/EvoNet-CNN-Insight/model_training/recent_moderate/RM


[I 2024-08-15 17:07:47,527] A new study created in RDB with name: no-name-7489b865-a029-4254-8484-a76d60185ac8


Trial 0 started with hyperparameters: num_layers=6, filters=[40, 109, 83, 35, 82, 126], kernel_size=3, l1=0.0025522613113152025, l2=4.5403543299974466e-05, dense_units=82, dropout_rate=0.49090520398674264, learning_rate=0.0009174143043032445
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 166ms/step - accuracy: 0.4790 - loss: 15.6271 - val_accuracy: 0.4825 - val_loss: 2.7800
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - accuracy: 0.5024 - loss: 1.7691 - val_accuracy: 0.4800 - val_loss: 0.8134
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - accuracy: 0.4967 - loss: 0.7919 - val_accuracy: 0.4725 - val_loss: 0.7715
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.5035 - loss: 0.7706 - val_accuracy: 0.4938 - val_loss: 0.7699
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - accuracy: 0.5086 - loss: 0.7700 - val_accuracy: 0.4913 - val_loss: 0.7693
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - accuracy: 0.4999 - loss: 0.7701 - val_accuracy: 0.5050 - val_loss: 0.7698
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.4937 

[I 2024-08-15 17:23:07,757] Trial 0 finished with value: 0.5 and parameters: {'num_layers': 6, 'filters_0': 40, 'filters_1': 109, 'filters_2': 83, 'filters_3': 35, 'filters_4': 82, 'filters_5': 126, 'kernel_size': 3, 'l1': 0.0025522613113152025, 'l2': 4.5403543299974466e-05, 'dense_units': 82, 'dropout_rate': 0.49090520398674264, 'learning_rate': 0.0009174143043032445}. Best is trial 0 with value: 0.5.


Trial 1 started with hyperparameters: num_layers=6, filters=[107, 59, 97, 119, 128, 46], kernel_size=2, l1=8.323494526900126e-06, l2=0.001266087065785262, dense_units=242, dropout_rate=0.4648593476000483, learning_rate=0.00048696206153286785
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 181ms/step - accuracy: 0.4970 - loss: 1.2345 - val_accuracy: 0.5175 - val_loss: 1.0053
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.4966 - loss: 0.9558 - val_accuracy: 0.4800 - val_loss: 0.8386
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.4965 - loss: 0.8142 - val_accuracy: 0.4725 - val_loss: 0.7590
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.5049 - loss: 0.7479 - val_accuracy: 0.5063 - val_loss: 0.7228
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.4923 - loss: 0.7180 - val_accuracy: 0.4913 - val_loss: 0.7070
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.4844 - loss: 0.7050 - val_accuracy: 0.5050 - val_loss: 0.7000
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.4837 -

...

Trial 48 started with hyperparameters: num_layers=5, filters=[39, 46, 58, 93, 57], kernel_size=3, l1=0.0012667763400342734, l2=0.003682685765450511, dense_units=139, dropout_rate=0.4845060484893591, learning_rate=0.00030795973582653806
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5125 - loss: 7.1020

[I 2024-08-15 22:39:40,914] Trial 48 pruned. Trial was pruned at epoch 0.


Trial 49 started with hyperparameters: num_layers=4, filters=[94, 40, 48, 40], kernel_size=2, l1=2.5482400563578544e-05, l2=0.00014165152095220067, dense_units=158, dropout_rate=0.3658281936168829, learning_rate=0.00014919084206189214
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.4937 - loss: 0.7630

[I 2024-08-15 22:41:20,505] Trial 49 pruned. Trial was pruned at epoch 0.


Best trial:
Value: 0.8489999771118164
Params: 
    num_layers: 4
    filters_0: 112
    filters_1: 70
    filters_2: 66
    filters_3: 97
    kernel_size: 3
    l1: 1.4140077757263846e-05
    l2: 6.14525850610673e-05
    dense_units: 136
    dropout_rate: 0.24035812561692615
    learning_rate: 0.000944861039761829
Elapsed time: 20013.26 seconds


Recent weak selection